In [2]:
# Installing the dependencies
import pandas as pd
import numpy as np

df = pd.read_csv('Book1.csv')
df



,age,income,student,credit rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31 . . .40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31 . . .40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [3]:
# A1. For the data provided below, calculate the prior probability for each class. 

# Data
data = [
    ['<=30', 'high', 'no', 'fair', 'no'],
    ['<=30', 'high', 'no', 'excellent', 'no'],
    ['31 . . .40', 'high', 'no', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'excellent', 'no'],
    ['31 . . .40', 'low', 'yes', 'excellent', 'yes'],
    ['<=30', 'medium', 'no', 'fair', 'no'],
    ['<=30', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'yes', 'fair', 'yes'],
    ['<=30', 'medium', 'yes', 'excellent', 'yes'],
    ['31 . . .40', 'medium', 'no', 'excellent', 'yes'],
    ['31 . . .40', 'high', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'excellent', 'no']
]

# Extract the "buys_computer" column
buys_computer_column = [row[-1] for row in data]

# Calculate the prior probabilities
total_instances = len(buys_computer_column)
buys_computer_no = buys_computer_column.count('no')
buys_computer_yes = buys_computer_column.count('yes')

# Calculate the prior probabilities
prior_no = buys_computer_no / total_instances
prior_yes = buys_computer_yes / total_instances

# Print the results
print(f'Prior probability for "buys_computer = no": {prior_no:.3f}')
print(f'Prior probability for "buys_computer = yes": {prior_yes:.3f}')



Prior probability for "buys_computer = no": 0.357
Prior probability for "buys_computer = yes": 0.643


In [4]:
# A2.Calculate the class conditional densities for various features & classes. Observe if any class 
# conditional density has zero values.


# Function to calculate class-conditional probabilities
def calculate_conditional_probs(feature, label, df):
    conditional_probs = {}
    classes = df[label].unique()

    for c in classes:
        class_data = df[df[label] == c]
        conditional_probs[c] = class_data[feature].value_counts(normalize=True).to_dict()

    return conditional_probs

# Calculate class-conditional probabilities for each feature
for feature in df.columns[:-1]:  # Exclude the target variable
    conditional_probs = calculate_conditional_probs(feature, 'buys_computer', df)
    
    print(f"Class-conditional densities for {feature}:")
    for label, probs in conditional_probs.items():
        print(f"P({feature} | buys_computer = {label}):")
        for value, prob in probs.items():
            print(f"  P({feature} = {value} | buys_computer = {label}): {prob:.4f}")
        print()


Class-conditional densities for age:
P(age | buys_computer = no):
  P(age = <=30 | buys_computer = no): 0.6000
  P(age = >40 | buys_computer = no): 0.4000

P(age | buys_computer = yes):
  P(age = 31 . . .40 | buys_computer = yes): 0.4444
  P(age = >40 | buys_computer = yes): 0.3333
  P(age = <=30 | buys_computer = yes): 0.2222

Class-conditional densities for income:
P(income | buys_computer = no):
  P(income = high | buys_computer = no): 0.4000
  P(income = medium | buys_computer = no): 0.4000
  P(income = low | buys_computer = no): 0.2000

P(income | buys_computer = yes):
  P(income = medium | buys_computer = yes): 0.4444
  P(income = low | buys_computer = yes): 0.3333
  P(income = high | buys_computer = yes): 0.2222

Class-conditional densities for student:
P(student | buys_computer = no):
  P(student = no | buys_computer = no): 0.8000
  P(student = yes | buys_computer = no): 0.2000

P(student | buys_computer = yes):
  P(student = yes | buys_computer = yes): 0.6667
  P(student = no 

In [5]:

# A3. Test for independence between the 4 given features.

from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(index=df['age'], columns=[df['income'], df['student'], df['credit rating']])

# Perform chi-square test
chi2, p, _, _ = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

# Interpret the results
alpha = 0.05
print(f"Significance level: {alpha}")
print(f"Is the p-value less than alpha? {'Yes' if p < alpha else 'No'}")

if p < alpha:
    print("Reject the null hypothesis. There is evidence of a significant association between the features.")
else:
    print("Fail to reject the null hypothesis. There is no significant evidence of an association between the features.")


Chi-square statistic: 12.95
P-value: 0.6764100579553458
Significance level: 0.05
Is the p-value less than alpha? No
Fail to reject the null hypothesis. There is no significant evidence of an association between the features.


In [6]:
# A4. Build a Naïve-Bayes (NB) classifier for the above given data. Below code for help.
# from sklearn.naive_bayes import GaussianNB
# model = GaussianNB()
# model.fit(Tr_X,Tr_y)

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report



# Encode categorical variables
label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

# Split the data into training and testing sets
X = df.drop('buys_computer', axis=1)
y = df['buys_computer']
Tr_X, Te_X, Tr_y, Te_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the Naïve-Bayes classifier
model = GaussianNB()
model.fit(Tr_X, Tr_y)

# Make predictions on the test set
predictions = model.predict(Te_X)

# Evaluate the model
accuracy = accuracy_score(Te_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(Te_y, predictions))


Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



C:\Users\PRADEEP\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PRADEEP\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PRADEEP\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [7]:
# A5. Build a NB classifier for your own project data.


In [18]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# Our Temporary Dataset( which we are using for Assignment Report)
df1 = pd.read_csv('./waterPotability_updated.csv')
print(df1.sample(20))



# Split the data into features (X) and target variable (y)
X = df1.drop('Potability', axis=1)
y = df1['Potability']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Train the classifier on the training set
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Print the accuracy of the classifier
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# You can also print other metrics like precision, recall, and F1 score if needed
print(metrics.classification_report(y_test, y_pred))


            ph    Hardness        Solids  Chloramines     Sulfate  \
1414  3.590822  213.702299  33431.671122     5.542364  308.380612   
2092  7.744099  219.658722  30809.705704     6.790681  333.775777   
1177  5.484151  196.791251  19378.359067     5.003731  295.591019   
427   9.879071  208.905645  14673.670669     6.240248  366.817639   
3104  8.631270  164.371169  14880.651798     7.278263  350.951581   
1530  7.080795  194.761215  38119.185568     6.998158  333.289216   
65    6.203978  212.306618  21815.074148     7.873992  333.775777   
3220  6.170526  193.335517  16206.219671     7.123966  333.775777   
1389  9.395842  162.157700   9225.770911     8.275035  333.775777   
2158  7.904053  217.771631  23575.991430     7.493024  279.146683   
2605  8.445219  228.522860  28966.569327     6.179855  333.775777   
2629  5.720136  182.440160  13463.852419     8.253966  333.775777   
297   7.080795  191.285218  21948.325362     5.626787  355.488766   
1077  8.037430  148.415309  48410.

In [15]:

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>Other Classifier We are using for the temporary Dataset<<<<<<<<<<<<<<<<<<

# 2.Decision Tree
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier


X = df1.drop('Potability', axis=1)  # Features
y = df1['Potability']  # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)  # 70% training and 30% testing

# Create a Decision Tree Classifier
clf = DecisionTreeClassifier()

# Train the model using the training sets
clf.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)

# Print classification report
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", metrics.recall_score(y_test, y_pred))

# Classification report for a more detailed performance analysis
print(classification_report(y_test, y_pred))

Accuracy: 0.6073245167853509
Precision: 0.5103092783505154
Recall: 0.5025380710659898
              precision    recall  f1-score   support

           0       0.67      0.68      0.67       589
           1       0.51      0.50      0.51       394

    accuracy                           0.61       983
   macro avg       0.59      0.59      0.59       983
weighted avg       0.61      0.61      0.61       983



In [10]:
# 3.Random Forest
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier
clf = RandomForestClassifier()

# Train the model using the training sets
clf.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)

# Print classification report
# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", metrics.recall_score(y_test, y_pred))

# Classification report for a more detailed performance analysis
print(classification_report(y_test, y_pred))

Accuracy: 0.6541200406917599
Precision: 0.6243093922651933
Recall: 0.2935064935064935
              precision    recall  f1-score   support

           0       0.66      0.89      0.76       598
           1       0.62      0.29      0.40       385

    accuracy                           0.65       983
   macro avg       0.64      0.59      0.58       983
weighted avg       0.65      0.65      0.62       983

